In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
from tqdm.notebook import tqdm
from gmsm.modules.models.msm import MSM
import time

root = Path().resolve().parent
data_path = root / 'data' / 'csi2007.csv'


In [ ]:
df = pd.read_csv(data_path, index_col=0)
df2007 = df[df['Year'] == 2007].loc[:, ['close']]

In [ ]:
df


In [ ]:
len(df2007)

In [ ]:
df.loc[:, 'log_returns'] = np.log(df['close']).diff()

In [ ]:
df_clean = df.dropna(subset=['log_returns'])
returns = df_clean['log_returns'].values

In [ ]:
kbar = 9
annualization_factor = len(df2007)
initial_estimation_years = 2
evaluation_start_year = 2008

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import warnings
from tqdm.auto import tqdm # More flexible tqdm import
import time # To time the process
import sys # Import sys for explicit exit
# Import traceback for detailed error info
import traceback

# --- Assume MSM class is defined above or imported ---
# >>>>>>>> PASTE YOUR COMPLETE MSM CLASS DEFINITION HERE <<<<<<<<<<<
# class MSM:
#    ... (Your full MSM class code) ...
# >>>>>>>> OR IMPORT IT: <<<<<<<<<<<
# from gmsm.modules.msm import MSM # Assuming your structure

# --- Configuration ---
# Path to your 5-minute data CSV
# >>>>>>>> IMPORTANT: Update this path <<<<<<<<<<<
root = Path().resolve().parent # Assuming script is one level down from project root
data_path = root / 'data' / 'csi2007.csv' # Path from user's code
data_file = data_path

# MSM Model Parameters
# --- FIX 1: Reduce KBAR significantly ---
KBAR = 3 # Start with 3 components (8 states), try 2 if this still fails
# --- FIX 2: Use standard annualization factor ---
ANNUALIZATION_FACTOR = 252 # Trading days in a year

# Rolling Window Parameters
INITIAL_ESTIMATION_YEARS = 3 # Years of data for the first fit
EVALUATION_START_YEAR = 2008 # Start forecasting from this year
MIN_ROLLING_WINDOW_DAYS = 200 # Minimum number of days required in the rolling window

# --- Helper Functions ---

def calculate_realized_volatility(df_5min_returns, annualization_factor=252):
    """Calculates daily realized variance and volatility from 5-min returns."""
    if not isinstance(df_5min_returns.index, pd.DatetimeIndex):
        raise ValueError("Input DataFrame must have a DatetimeIndex.")
    df_copy = df_5min_returns.copy()
    # Ensure log returns are numeric and handle potential infinities
    df_copy['log_ret_5min'] = pd.to_numeric(df_copy['log_ret_5min'], errors='coerce')
    df_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_copy.dropna(subset=['log_ret_5min'], inplace=True) # Drop NaNs again after coercion/inf handling

    df_copy['log_ret_5min_sq'] = df_copy['log_ret_5min']**2
    # Ensure sum only happens if there are valid squared returns for the day
    realized_variance_daily = df_copy['log_ret_5min_sq'].resample('D').sum(min_count=1) # Require at least 1 valid sq return
    realized_volatility_daily = np.sqrt(realized_variance_daily * annualization_factor)
    realized_volatility_daily.name = 'realized_volatility'
    return realized_volatility_daily

def calculate_daily_returns(df_5min, price_col='close'):
    """Calculates daily log returns from 5-min close prices."""
    if not isinstance(df_5min.index, pd.DatetimeIndex):
        raise ValueError("Input DataFrame must have a DatetimeIndex.")
    # Ensure price column is numeric
    df_5min[price_col] = pd.to_numeric(df_5min[price_col], errors='coerce')
    # Use 'B' frequency to only get business day closes
    daily_close = df_5min[price_col].resample('B').last()
    # Forward fill missing daily closes (e.g., holidays) before calculating returns
    daily_close = daily_close.ffill()
    daily_log_returns = np.log(daily_close).diff()
    daily_log_returns.name = 'log_ret_daily'
    return daily_log_returns

def evaluate_forecasts(comparison_df):
    """Calculates and prints evaluation metrics."""
    required_cols = ['realized_volatility', 'msm_forecast']
    if not all(col in comparison_df.columns for col in required_cols):
        print("Error: Missing required columns for evaluation.")
        return None
    eval_df = comparison_df[required_cols].dropna()
    if eval_df.empty or len(eval_df) < 2: # Need at least 2 points for regression
        print("No overlapping data or insufficient data available for evaluation after dropping NaNs.")
        return None

    realized_vol = eval_df['realized_volatility']
    forecast_vol = eval_df['msm_forecast']

    # Add check for non-finite values before metrics
    finite_mask = np.isfinite(realized_vol) & np.isfinite(forecast_vol)
    if not finite_mask.all():
        print("Warning: Non-finite values found in realized or forecast volatility. Filtering before evaluation.")
        realized_vol = realized_vol[finite_mask]
        forecast_vol = forecast_vol[finite_mask]
        if len(realized_vol) < 2:
             print("Insufficient finite data points remaining for evaluation.")
             return None


    # --- Metrics ---
    if len(realized_vol) == 0 or len(forecast_vol) == 0:
        print("Cannot calculate metrics: No valid data points.")
        return eval_df

    try:
        mse = mean_squared_error(realized_vol, forecast_vol)
        mae = np.mean(np.abs(realized_vol - forecast_vol))
        rmse = np.sqrt(mse)
        epsilon = 1e-12
        realized_var = np.maximum(realized_vol, 0)**2 + epsilon
        forecast_var = np.maximum(forecast_vol, 0)**2 + epsilon
        forecast_var = np.maximum(forecast_var, epsilon)
        valid_qlike_mask = (forecast_var > 0) & (realized_var >= 0)
        if np.sum(valid_qlike_mask) > 0:
            qlike = np.mean(np.log(forecast_var[valid_qlike_mask]) + realized_var[valid_qlike_mask] / forecast_var[valid_qlike_mask])
        else:
            qlike = np.nan

        print("\n--- Forecast Evaluation Metrics ---")
        print(f"  RMSE: {rmse:.6f}")
        print(f"  MAE:  {mae:.6f}")
        print(f"  MSE:  {mse:.6f}")
        print(f"  QLIKE:{qlike:.4f} (Lower is better)")

    except Exception as metric_e:
        print(f"Error calculating basic metrics: {metric_e}")
        return eval_df


    # --- Mincer-Zarnowitz Regression ---
    if np.std(forecast_vol) > 1e-9 and len(forecast_vol) > 1:
        X = sm.add_constant(forecast_vol)
        y = realized_vol
        try:
            mz_model = sm.OLS(y, X).fit()
            print("\n--- Mincer-Zarnowitz Regression (Realized ~ const + Forecast) ---")
            print(mz_model.summary())
        except Exception as e:
            print(f"\nCould not perform Mincer-Zarnowitz regression: {e}")
    else:
        print("\nSkipping Mincer-Zarnowitz regression: Insufficient variation in forecasts or too few data points.")


    # --- Plotting ---
    try:
        plt.figure(figsize=(14, 7))
        # Use original eval_df index but filtered data for plotting
        plot_index = eval_df.index[finite_mask] if 'finite_mask' in locals() and finite_mask.any() else eval_df.index
        plt.plot(plot_index, realized_vol, label='Realized Volatility (Ground Truth)', alpha=0.7, linewidth=1.5, color='black')
        plt.plot(plot_index, forecast_vol, label=f'MSM Forecast (k={KBAR})', alpha=0.8, linewidth=1.2, color='red')
        plt.title('MSM Daily Volatility Forecast vs. Realized Volatility')
        plt.xlabel('Date')
        plt.ylabel('Annualized Volatility')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()
    except Exception as plot_e:
        print(f"Error during plotting: {plot_e}")


    return eval_df

# --- Main Pipeline ---

# 1. Load and Prepare 5-minute Data
print(f"Loading 5-minute data from: {data_file}...")
try:
    df_5min = pd.read_csv(
        data_file,
        index_col='datetime',
        parse_dates=True,
    )
    if df_5min.empty: print(f"Error: DataFrame is empty after loading data from {data_file}."); sys.exit(1)
    df_5min.sort_index(inplace=True)
    print(f"Loaded {len(df_5min)} records.")
    print(f"Data range: {df_5min.index.min()} to {df_5min.index.max()}")
    required_cols_5min = ['close']
    if not all(col in df_5min.columns for col in required_cols_5min):
         missing_cols = [col for col in required_cols_5min if col not in df_5min.columns]
         print(f"Error: Missing required columns in loaded 5-min data: {missing_cols}"); sys.exit(1)
except FileNotFoundError: print(f"Error: Data file not found at {data_file}"); raise
except Exception as e: print(f"Error loading or performing initial checks on data: {e}"); raise

# 2. Calculate 5-min Returns
print("Calculating 5-minute log returns...")
if 'close' not in df_5min.columns: print("Error: 'close' column not found."); sys.exit(1)
df_5min['close'] = pd.to_numeric(df_5min['close'], errors='coerce')
df_5min.dropna(subset=['close'], inplace=True)
df_5min['log_ret_5min'] = np.log(df_5min['close']).diff()
df_5min = df_5min.dropna(subset=['log_ret_5min'])

# 3. Calculate Daily Realized Volatility (Ground Truth)
print("Calculating daily realized volatility...")
daily_rv = calculate_realized_volatility(df_5min, ANNUALIZATION_FACTOR)

# 4. Calculate Daily Returns (for MSM fitting)
print("Calculating daily log returns...")
daily_returns = calculate_daily_returns(df_5min)

# 5. Align Data into a Daily DataFrame
print("Aligning daily data...")
df_daily = pd.DataFrame(index=daily_returns.index)
df_daily = df_daily.join(daily_returns).join(daily_rv)
df_daily = df_daily.dropna(subset=['log_ret_daily', 'realized_volatility'])
df_daily = df_daily[np.isfinite(df_daily['log_ret_daily'])]
print(f"Created daily DataFrame with {len(df_daily)} trading days.")
print(f"Daily data range: {df_daily.index.min().date()} to {df_daily.index.max().date()}")

# 6. Rolling Window Forecasting Setup
print("\nSetting up rolling window forecasting...")
try:
    initial_estimation_start_date = df_daily.index.min()
    initial_estimation_end_date_target = initial_estimation_start_date + pd.DateOffset(years=INITIAL_ESTIMATION_YEARS) - pd.Timedelta(days=1)
    valid_end_dates = df_daily.index[df_daily.index <= initial_estimation_end_date_target]
    if valid_end_dates.empty: raise ValueError("Cannot determine initial estimation end date.")
    initial_estimation_end_date = valid_end_dates[-1]

    initial_window_data = df_daily.loc[initial_estimation_start_date:initial_estimation_end_date]
    initial_window_returns = initial_window_data['log_ret_daily'].dropna().values
    initial_window_size_days = len(initial_window_returns) # Use actual returns count
    print(f"Initial estimation window size: {initial_window_size_days} days")

    eval_start_lookup = df_daily.index[df_daily.index >= f"{EVALUATION_START_YEAR}-01-01"]
    if eval_start_lookup.empty: raise ValueError(f"No data found starting from {EVALUATION_START_YEAR}")
    evaluation_start_date = eval_start_lookup[0]
    evaluation_dates = df_daily.loc[evaluation_start_date:].index
    print(f"Initial estimation period: {initial_estimation_start_date.date()} to {initial_estimation_end_date.date()}")
    print(f"Evaluation period: {evaluation_start_date.date()} to {df_daily.index.max().date()} ({len(evaluation_dates)} days)")

    if initial_window_size_days < MIN_ROLLING_WINDOW_DAYS:
        print(f"Warning: Initial estimation window ({initial_window_size_days} days) is smaller than MIN_ROLLING_WINDOW_DAYS ({MIN_ROLLING_WINDOW_DAYS}).")

except (IndexError, ValueError) as e: print(f"Error setting up estimation/evaluation periods: {e}. Check data range and EVALUATION_START_YEAR."); raise
except Exception as e: print(f"Unexpected error setting up estimation/evaluation periods: {e}"); raise

# --- DEBUG: Test Fit on Initial Window ---
print("\n--- Attempting fit on INITIAL estimation window ---")
if len(initial_window_returns) >= MIN_ROLLING_WINDOW_DAYS and np.all(np.isfinite(initial_window_returns)):
    try:
        print(f"Fitting MSM(k={KBAR}) on initial window ({len(initial_window_returns)} days) with verbose=True...")
        initial_model = MSM(ret=initial_window_returns, kbar=KBAR, n_vol=ANNUALIZATION_FACTOR, verbose=True)
        if initial_model.parameters is None or not initial_model.results.get('optim_convergence', False):
            print("--- Initial Fit FAILED or did not converge ---")
            print(f"   Convergence: {initial_model.results.get('optim_convergence', 'N/A')}")
            print(f"   Message: {initial_model.results.get('optim_message', 'N/A')}")
        else:
            print("--- Initial Fit SUCCESSFUL ---")
            print(f"   LogLik: {initial_model.log_likelihood:.4f}")
            print(f"   Params (unannualized sigma): {initial_model.parameters}")
            # Optionally try a prediction
            # initial_pred = initial_model.predict(h=1)
            # print(f"   Initial 1-step prediction: {initial_pred}")
    except Exception as initial_e:
        print(f"--- ERROR during initial fit attempt: {type(initial_e).__name__} - {initial_e} ---")
        print(traceback.format_exc()) # Print full traceback for the initial fit error
else:
    print(f"--- Skipping initial fit test: Window size {len(initial_window_returns)} < {MIN_ROLLING_WINDOW_DAYS} or non-finite data ---")
# --- End DEBUG Block ---


print("\nStarting rolling window forecasting loop...")
msm_forecasts = []
forecast_dates = []
failed_fits = 0 # Counter for failed MSM fits
start_time = time.time()

# Use tqdm for progress bar
for i in tqdm(range(len(evaluation_dates)), desc="Rolling Forecast"):
    forecast_for_date = evaluation_dates[i]
    forecast_value = np.nan # Default forecast value if anything fails

    try:
        # --- Window Selection ---
        window_end_idx_loc = df_daily.index.get_loc(forecast_for_date) - 1
        if window_end_idx_loc < 0: continue

        window_start_idx_loc = max(0, window_end_idx_loc - initial_window_size_days + 1)
        current_window_data = df_daily.iloc[window_start_idx_loc : window_end_idx_loc + 1]
        current_returns = current_window_data['log_ret_daily'].dropna().values

        # --- Check Window Size ---
        if len(current_returns) < MIN_ROLLING_WINDOW_DAYS:
            msm_forecasts.append(np.nan)
            forecast_dates.append(forecast_for_date)
            continue

        # --- Attempt MSM Fit ---
        rolling_model = None
        if not np.all(np.isfinite(current_returns)):
             failed_fits += 1
             forecast_value = np.nan
        else:
            # --- Fit Model (verbose=False in loop) ---
            rolling_model = MSM(ret=current_returns, kbar=KBAR, n_vol=ANNUALIZATION_FACTOR, verbose=False)

            # --- Check Fit Success ---
            if rolling_model.parameters is None or not rolling_model.results.get('optim_convergence', False):
                failed_fits += 1
                forecast_value = np.nan
            else:
                # --- Attempt Prediction ---
                prediction = rolling_model.predict(h=1)
                if prediction and 'vol' in prediction and not np.isnan(prediction['vol'][0, 0]):
                    forecast_value = prediction['vol'][0, 0]
                else:
                    forecast_value = np.nan # Ensure NaN if predict fails

    except Exception as e:
        # --- Log the specific error for this iteration ---
        # Only print first few errors to avoid flooding output
        if failed_fits < 10:
             print(f"\nERROR during loop for {forecast_for_date.date()}: {type(e).__name__} - {e}")
             # Optionally print traceback for first error
             # if failed_fits == 0:
             #    traceback.print_exc()
        elif failed_fits == 10:
             print("\n...(suppressing further error messages from loop)...")

        failed_fits += 1
        forecast_value = np.nan

    # --- Store Results for this date ---
    msm_forecasts.append(forecast_value)
    forecast_dates.append(forecast_for_date)


end_time = time.time()
print(f"\nRolling forecast loop finished in {end_time - start_time:.2f} seconds.")
if failed_fits > 0:
    print(f"Warning: The MSM model fit failed, did not converge, or encountered an error for {failed_fits} out of {len(evaluation_dates)} windows.")

# 7. Combine Forecasts and Evaluate
print("Combining forecasts with daily data...")
if not forecast_dates: print("Error: No forecast dates were generated."); sys.exit(1)
forecast_series = pd.Series(msm_forecasts, index=pd.Index(forecast_dates, name='datetime'), name='msm_forecast')
if forecast_series.index.has_duplicates:
    print("Warning: Duplicate forecast dates found. Keeping last."); forecast_series = forecast_series[~forecast_series.index.duplicated(keep='last')]
df_daily_eval = pd.merge(df_daily, forecast_series, left_index=True, right_index=True, how='left')
df_eval_period = df_daily_eval.loc[evaluation_start_date:].copy()
valid_forecast_count = df_eval_period['msm_forecast'].count()
print(f"Generated {valid_forecast_count} valid forecasts out of {len(df_eval_period)} evaluation days.")

if valid_forecast_count > 0:
    evaluation_results_df = evaluate_forecasts(df_eval_period)
    if evaluation_results_df is not None: print("\n--- Sample of Forecasts vs. Realized Volatility ---\n", evaluation_results_df.head())
    else: print("\nEvaluation could not be completed.")
else: print("\nNo valid forecasts were generated, skipping evaluation.")

print("\nPipeline finished.")

In [ ]:
# Necessary Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import stats
import warnings
from pathlib import Path
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm # More flexible tqdm import
import time # To time the process
import sys # Import sys for explicit exit
import traceback # Import traceback for detailed error info

# =============================================================================
# Markov Switching Multifractal Model (MSM) Class Definition
# =============================================================================
class MSM:
    """
    MSM from the 2004 Calvert & Fisher Paper
    -----------
    ret : numpy.ndarray
        Input time series of returns (T x 1).
    kbar : int
        Number of frequency components (volatility cascades).
    n_vol : int
        Number of trading periods in a year (for annualization).
    nw_lag : int
        Number of lags for Newey-West standard errors (currently affects gradient calc).
    parameters : numpy.ndarray
        Estimated model parameters [m0, b, gamma_k, sigma_unannualized].
    std_errors : numpy.ndarray
        Standard errors of the estimated parameters.
    log_likelihood : float
        The maximized log-likelihood value of the fitted model.
    results : dict
        - LL: Log-likelihood.
        - LLs: Vector of log-likelihood contributions per time step.
        - filtered_probabilities: P(state_t | data_{1:t}).
        - smoothed_probabilities: P(state_t | data_{1:T}).
        - transition_matrix: Estimated state transition matrix A.
        - state_vol_multipliers: Estimated state volatility multipliers g_m.
        - component_matrix: Matrix mapping states to component values (Mmat).
        - optim_message: Message from the optimizer.
        - optim_convergence: Boolean indicating optimizer convergence.
        - optim_iter: Number of optimizer iterations.
        - parameters: Estimated parameters [m0, b, gamma_k, sigma_unannualized].
        - std_errors: Standard errors.
        - coefficients: Coefficients ready for display (b is NaN if kbar=1, sigma is annualized).
    """
    def __init__(self, ret, kbar=1, n_vol=252, para0=None, nw_lag=0):
        """
        Parameters:
        -----------
        ret : array-like
            Vector or Series of financial returns.
        kbar : int, optional
            Number of frequency components (volatility cascades), default is 1.
            Determines the number of states (2^kbar).
        n_vol : int, optional
            Number of trading periods in a year (e.g., 252 for daily data),
            used for annualizing sigma. Default is 252.
        para0 : list or tuple, optional
            Initial parameter values [m0, b, gammak, sigma_annualized] for optimization.
            If None, default starting values are used.
        nw_lag : int, optional
            Number of lags for Newey-West adjustment in standard error calculation.
            Default is 0 (no adjustment). Note: Implementation currently uses OPG standard errors.
        """
        # Store configuration
        self.n_vol = n_vol
        self.nw_lag = nw_lag

        # Validate and prepare inputs
        checked_inputs = self._check_and_prepare_inputs(ret, kbar, para0)
        self.ret = checked_inputs["dat"]
        self.kbar = checked_inputs["kbar"]
        self.k_states = 2**self.kbar
        self._para0 = checked_inputs["start_value"] # Initial guess (sigma annualized)
        self._lb = checked_inputs["lb"] # Lower bounds (sigma unannualized)
        self._ub = checked_inputs["ub"] # Upper bounds (sigma unannualized)

        # De-annualize initial sigma guess for internal use
        self._para0[3] = self._para0[3] / np.sqrt(self.n_vol)

        # Initialize results attributes
        self.parameters = None
        self.std_errors = None
        self.log_likelihood = None
        self.results = {}

        # Fit the model upon initialization
        # Use try-except for robustness during initialization
        try:
            self._fit() # <--- Model fitting happens here
        except Exception as e:
            warnings.warn(f"Error during initial _fit call: {e}", RuntimeWarning)
            # Ensure attributes are set even if fit fails
            self.parameters = None
            self.std_errors = None
            self.log_likelihood = -np.inf
            self.results = {"optim_convergence": False, "optim_message": f"Error in _fit: {e}"}


    def _matrix_power(self, A, power):
        """
        Calculate matrix power using numpy's linalg.matrix_power.
        Ensures power is a non-negative integer and A is square.
        """
        if not isinstance(power, int) or power < 0:
            raise ValueError("power must be a non-negative integer.")
        if A.ndim != 2 or A.shape[0] != A.shape[1]:
            raise ValueError("A must be a square matrix.")
        # Use numpy's efficient matrix power function
        return np.linalg.matrix_power(A, power)

    def _check_and_prepare_inputs(self, dat, kbar, x0):
        """
        Validate input data and parameters, return processed inputs.
        Handles pandas Series/DataFrame, ensures returns are a column vector,
        checks for NaNs, validates kbar, sets parameter bounds, and prepares
        initial parameter guesses (para0).
        """
        # Convert pandas input to numpy array
        if isinstance(dat, (pd.DataFrame, pd.Series)):
            dat = dat.values

        # Ensure data is a numpy array
        if not isinstance(dat, np.ndarray):
            dat = np.array(dat)

        # Ensure data is a column vector (T x 1)
        if dat.ndim == 1:
            dat = dat.reshape(-1, 1)
        elif dat.shape[1] > 1:
            # Warn if multiple columns, use the first one
            warnings.warn("Input data has multiple columns. Using the first column for returns.", UserWarning)
            dat = dat[:, 0].reshape(-1, 1)

        # Check for NaN values in the final return vector
        if np.any(np.isnan(dat)):
            raise ValueError("Input data contains NaN values. Please remove them before passing to the MSM class.")

        # Validate kbar
        if kbar < 1 or not isinstance(kbar, int):
            raise ValueError('kbar (number of volatility components) must be a positive integer.')

        # Define parameter bounds [m0, b, gammak, sigma_unannualized]
        # m0: 1 < m0 < 2
        # b: b > 1 (only relevant if kbar > 1)
        # gammak: 0 < gammak < 1
        # sigma: sigma > 0
        lb = [1.0 + 1e-6, 1.0 + 1e-6, 1e-6, 1e-6]
        ub = [2.0 - 1e-6, 50.0, 1.0 - 1e-6, 50.0] # Allow large b and sigma

        # Process initial parameter guess (para0) if provided
        if x0 is not None:
            if len(x0) != 4:
                raise ValueError('Initial values (para0) must be of length 4: [m0, b, gammak, sigma_annualized]')

            m0, b_init, gamma_k_init, sigma_ann_init = x0

            # Check and clamp initial values to bounds
            if not (lb[0] <= m0 <= ub[0]):
                warnings.warn(f"Initial m0 ({m0}) outside bounds. Clamping.", UserWarning)
                m0 = np.clip(m0, lb[0], ub[0])
            # Parameter b is only relevant if kbar > 1
            if kbar > 1 and not (lb[1] <= b_init <= ub[1]):
                warnings.warn(f"Initial b ({b_init}) outside bounds. Clamping.", UserWarning)
                b_init = np.clip(b_init, lb[1], ub[1])
            elif kbar == 1:
                b_init = 1.5 # Assign default if kbar=1 (won't be used)
            if not (lb[2] <= gamma_k_init <= ub[2]):
                warnings.warn(f"Initial gammak ({gamma_k_init}) outside bounds. Clamping.", UserWarning)
                gamma_k_init = np.clip(gamma_k_init, lb[2], ub[2])
            # Ensure initial sigma is positive
            if sigma_ann_init <= 0:
                warnings.warn(f"Initial annualized sigma ({sigma_ann_init}) must be positive. Using data std dev.", UserWarning)
                # Calculate default using data std dev
                sigma_ann_def_calc = np.std(dat) * np.sqrt(self.n_vol)
                # Use calculated std dev or slightly above lower bound if std dev is too small
                sigma_ann_init = max(sigma_ann_def_calc, lb[3] * np.sqrt(self.n_vol) * 1.01)

            start_value = [m0, b_init, gamma_k_init, sigma_ann_init] # Sigma is annualized here

        else:
            # Define default initial values if none provided
            m0_def = 1.5
            b_def = 2.5 if kbar > 1 else 1.5 # Default b > 1 only if needed
            gamma_k_def = 0.9
            # Calculate annualized std dev from data as initial sigma guess
            sigma_ann_def = np.std(dat) * np.sqrt(self.n_vol)
            # Ensure default sigma is positive and respects lower bound
            # Compare annualized default to annualized version of lower bound
            if sigma_ann_def <= (lb[3] * np.sqrt(self.n_vol)):
                 sigma_ann_def = lb[3] * np.sqrt(self.n_vol) * 1.1 # Use lower bound (annualized) if std dev is too small

            start_value = [m0_def, b_def, gamma_k_def, sigma_ann_def] # Sigma is annualized here
            # print(f"Using default starting values (annualized sigma): {start_value}") # Removed verbose flag dependency

        # Adjust bounds and start value for 'b' if kbar == 1 (parameter b is irrelevant)
        if kbar == 1:
            lb[1] = 1.5  # Fix b to an arbitrary valid value within bounds
            ub[1] = 1.5
            start_value[1] = 1.5  # Fix initial b as well

        return {
            "dat": dat,
            "kbar": kbar,
            "start_value": start_value,  # sigma is annualized here
            "lb": lb,  # sigma bound is unannualized
            "ub": ub  # sigma bound is unannualized
        }

    def _calculate_transition_matrix(self, b, gamma_kbar):
        """
        Calculates the state transition matrix A based on parameters b and gamma_kbar.
        Handles clipping of parameters to ensure numerical stability.
        Uses the Kronecker product to build the full matrix for kbar > 1.
        Normalizes rows to sum to 1.
        """
        # Clip parameters within the function to ensure validity for calculations
        gamma_kbar = np.clip(gamma_kbar, self._lb[2], self._ub[2])
        if self.kbar > 1:
             b = np.clip(b, self._lb[1], self._ub[1])
        else:
             b = 1.5 # Set b explicitly if kbar=1

        # Initialize gamma_k vector (transition probabilities for each component)
        gamma_k = np.zeros(self.kbar)

        # Calculate gamma_1 based on gamma_kbar and b
        # Avoid log(<=0) or 0^negative_power issues
        power_base = max(1.0 - gamma_kbar, 1e-10)
        exponent = 1.0 / (b**(self.kbar - 1)) if self.kbar > 1 else 1.0
        try:
            # Use np.power for potentially fractional exponents safely
            gamma_k[0] = 1.0 - np.power(power_base, exponent)
        except ValueError: # Handle potential complex result if base is negative (highly unlikely with clip)
            gamma_k[0] = 1e-10 # Assign small positive value
            warnings.warn("Numerical issue calculating gamma_k[0]. Setting to small value.", RuntimeWarning)

        # Ensure gamma_k[0] is within valid probability range [epsilon, 1-epsilon]
        gamma_k[0] = np.clip(gamma_k[0], 1e-10, 1.0 - 1e-10)

        # Base 2x2 transition matrix for the first component (k=1)
        # A_comp = [[P(0->0), P(0->1)], [P(1->0), P(1->1)]]
        # P(0->1) = P(1->0) = 0.5 * gamma_k
        # P(0->0) = P(1->1) = 1 - 0.5 * gamma_k
        A_comp1 = np.array([
            [1.0 - 0.5 * gamma_k[0], 0.5 * gamma_k[0]],
            [0.5 * gamma_k[0], 1.0 - 0.5 * gamma_k[0]]
        ])
        A = A_comp1 # Start with the first component's matrix

        # Build the full transition matrix using Kronecker product for kbar > 1
        if self.kbar > 1:
            # Calculate remaining gamma_k values
            base_gamma0 = max(1.0 - gamma_k[0], 1e-10) # Use gamma_1 (gamma_k[0])
            for i in range(1, self.kbar):
                try:
                    # gamma_k[i] = 1 - (1 - gamma_k[0])**(b**i)
                    gamma_k[i] = 1.0 - np.power(base_gamma0, (b ** i))
                except ValueError:
                    gamma_k[i] = 1e-10
                    warnings.warn(f"Numerical issue calculating gamma_k[{i}]. Setting to small value.", RuntimeWarning)

                # Clip subsequent gamma_k values
                gamma_k[i] = np.clip(gamma_k[i], 1e-10, 1.0 - 1e-10)

                # Transition matrix for component i+1
                a_i = np.array([
                    [1.0 - 0.5 * gamma_k[i], 0.5 * gamma_k[i]],
                    [0.5 * gamma_k[i], 1.0 - 0.5 * gamma_k[i]]
                ])
                # Combine with existing matrix using Kronecker product
                A = np.kron(A, a_i)

        # Normalize rows to ensure they sum exactly to 1 (handle potential floating point inaccuracies)
        row_sums = np.sum(A, axis=1, keepdims=True)
        # Avoid division by zero or near-zero
        row_sums[row_sums <= 1e-10] = 1.0
        A = A / row_sums
        # Clip again to handle any minor numerical errors pushing values outside [0, 1]
        A = np.clip(A, 0, 1)
        # Re-normalize as a final safety check
        row_sums = np.sum(A, axis=1, keepdims=True)
        row_sums[row_sums <= 1e-10] = 1.0
        A = A / row_sums

        return A

    def _calculate_component_matrix(self, m0):
        """
        Calculates the Mmat matrix (k_states x kbar).
        Mmat[i, k] gives the value (m0 or m1=2-m0) of the (k+1)-th
        volatility component when the system is in state i.
        State i is interpreted as a binary number (b_kbar ... b_1),
        where b_k=1 means component k is in the high-volatility state (m1).
        """
        # Clip m0 to ensure m0 and m1 are valid multipliers
        m0 = np.clip(m0, self._lb[0], self._ub[0])
        m1 = 2.0 - m0
        Mmat = np.zeros((self.k_states, self.kbar))

        # Iterate through each state (rows)
        for i in range(self.k_states):
            # Iterate through each component (columns)
            for j in range(self.kbar):
                # Check the j-th bit of the state index i
                # If the j-th bit is 1, component j+1 has value m1
                # If the j-th bit is 0, component j+1 has value m0
                if (i >> j) & 1:
                    Mmat[i, j] = m1
                else:
                    Mmat[i, j] = m0
        return Mmat

    def _calculate_state_vol_multipliers(self, m0):
        """
        Calculates the volatility multiplier g_m for each state.
        g_m = sqrt(product of component values M_k for that state).
        Uses the _calculate_component_matrix method.
        Returns a row vector (1 x k_states).
        """
        # Clip m0 first
        m0 = np.clip(m0, self._lb[0], self._ub[0])

        # Get the matrix of component values for each state
        Mmat = self._calculate_component_matrix(m0) # Shape (k_states x kbar)

        # Calculate the product of component values along axis 1 (across components) for each state
        g_m_sq = np.prod(Mmat, axis=1) # Shape (k_states,)

        # Ensure non-negativity before taking the square root (should be guaranteed by m0 bounds)
        g_m_sq = np.maximum(g_m_sq, 1e-16) # Use a small floor > 0
        g_m = np.sqrt(g_m_sq)

        # Return as a row vector (1 x k_states)
        return g_m.reshape(1, -1)

    def _calculate_conditional_densities(self, params):
        """
        Calculates the conditional probability densities p(ret_t | state_j)
        for all time steps t and all states j. Assumes returns are normally
        distributed with mean 0 and state-dependent variance.
        Uses scipy.stats.norm for calculation.
        Returns omega_t matrix (T x k_states).
        """
        # Extract necessary parameters (m0 for multipliers, sigma_unann for scale)
        m0, _, _, sigma_unann = params
        # Calculate state-dependent volatility multipliers
        g_m = self._calculate_state_vol_multipliers(m0)  # Shape (1, k_states)

        # Clip sigma_unann to ensure positivity
        sigma_unann = np.maximum(sigma_unann, 1e-8)

        # Calculate state-dependent standard deviations (unannualized)
        state_sigmas = sigma_unann * g_m  # Shape (1, k_states)

        # Prepare matrices for vectorized calculation
        T = self.ret.shape[0]
        # Tile state sigmas T times vertically -> (T, k_states)
        sig_mat = np.tile(state_sigmas, (T, 1))
        # Tile returns k_states times horizontally -> (T, k_states)
        ret_mat = np.tile(self.ret, (1, self.k_states))

        # Avoid division by zero or very small sigma in PDF calculation
        sig_mat = np.maximum(sig_mat, 1e-16) # Floor sigma values

        # Calculate normal PDF using scipy.stats.norm(x, loc=mean, scale=std_dev)
        try:
            omega_t = stats.norm.pdf(ret_mat, loc=0, scale=sig_mat)
        except Exception as e:
             # Fallback to manual calculation if stats.norm fails (unlikely)
             warnings.warn(f"Scipy stats.norm.pdf failed: {e}. Using manual calculation.", RuntimeWarning)
             norm_const = 1.0 / (np.sqrt(2 * np.pi) * sig_mat)
             exponent = -0.5 * np.square(ret_mat / sig_mat)
             # Handle potential overflow in exp
             exponent = np.clip(exponent, -700, 700)
             omega_t = norm_const * np.exp(exponent)

        # Floor the densities to prevent exact zeros (important for log-likelihood)
        omega_t = np.maximum(omega_t, 1e-300) # Use a very small floor

        return omega_t

    def _calculate_likelihood(self, params, return_details=False):
        """
        Calculates the log-likelihood of the model using the Hamilton filter.
        Iterates through time, calculating predicted state probabilities,
        likelihood of the observation, and updating filtered probabilities.

        Parameters:
        -----------
        params : array-like [m0, b, gammak, sigma_unannualized]
        return_details : bool, If True, returns dict with filter details.

        Returns:
        --------
        float (negative log-likelihood) or dict (if return_details=True).
        Handles numerical issues by returning large penalty if errors occur.
        """
        # Clip parameters at the start to ensure they are within bounds
        params_clipped = np.clip(params, self._lb, self._ub)
        # Special handling for b if kbar=1 (it's fixed during optimization)
        if self.kbar == 1:
             params_clipped[1] = 1.5

        m0, b, gamma_k, sigma_unann = params_clipped
        T_obs = self.ret.shape[0]
        k_st = self.k_states

        # --- Calculate model components (A, g_m, omega_t) ---
        try:
            A = self._calculate_transition_matrix(b, gamma_k)
            g_m = self._calculate_state_vol_multipliers(m0)
            omega_t = self._calculate_conditional_densities(params_clipped)
        except Exception as e:
            # If component calculation fails, return large penalty
            warnings.warn(f"Error during component calculation in likelihood: {e}. Returning large neg LL.", RuntimeWarning)
            fallback_return = {"pmat": np.ones((T_obs + 1, k_st)) / k_st, "LL": 1e12, "LLs": np.full(T_obs, -1e12 / T_obs), "A": np.eye(k_st), "g_m": np.ones((1, k_st))}
            return fallback_return if return_details else 1e12

        # --- Hamilton Filter Initialization ---
        pmat = np.zeros((T_obs + 1, k_st)) # Filtered probs P(S_t | data_{1:t}), row 0 is initial P(S_0)
        LLs = np.zeros(T_obs)              # Log-likelihood contributions log p(y_t | Y_{t-1})

        # Initialize P(S_0) with stationary distribution of A if possible, else uniform
        try:
            eigenvalues, eigenvectors = np.linalg.eig(A.T)
            stationary_dist_idx = np.isclose(eigenvalues, 1.0)
            if np.sum(stationary_dist_idx) > 0:
                # Take the real part of the eigenvector corresponding to eigenvalue 1
                stat_dist = eigenvectors[:, stationary_dist_idx].real[:,0]
                # Normalize to sum to 1
                pmat[0, :] = stat_dist / np.sum(stat_dist)
            else: # Fallback if no eigenvalue close to 1 found
                 pmat[0, :] = 1.0 / k_st
        except np.linalg.LinAlgError: # Fallback for eig decomposition failure
            pmat[0, :] = 1.0 / k_st

        # Ensure initial distribution is valid (non-negative, sums to 1)
        pmat[0, :] = np.maximum(pmat[0, :], 0)
        pmat[0, :] = pmat[0, :] / np.sum(pmat[0, :])

        # --- Hamilton Filter Loop ---
        log_lik_floor = -700 # Approx log(1e-304), floor for log-likelihood contribution

        for t in range(T_obs):
            # Predict step: P(S_t | data_{1:t-1}) = sum_i P(S_{t-1}=i | data_{1:t-1}) * P(S_t | S_{t-1}=i)
            # Ensure pmat[t,:] (P(S_{t-1}|data_{1:t-1})) is valid before matmul
            if np.any(np.isnan(pmat[t, :])) or np.sum(pmat[t,:]) <= 1e-10:
                 pmat[t, :] = 1.0 / k_st # Reset if invalid state

            predicted_prob_st = pmat[t, :] @ A  # Shape (1, k_states)
            # Clip predicted probs for numerical stability
            predicted_prob_st = np.maximum(predicted_prob_st, 0)
            sum_pred = np.sum(predicted_prob_st)
            # Normalize predicted probability
            if sum_pred > 1e-10:
                 predicted_prob_st = predicted_prob_st / sum_pred
            else:
                 predicted_prob_st = np.ones(k_st) / k_st # Fallback if prediction is zero

            # Likelihood of observation y_t: p(y_t | data_{1:t-1}) = sum_j P(S_t=j | data_{1:t-1}) * p(y_t | S_t=j)
            # omega_t[t, :] is p(y_t | S_t=j) for j=0..k_states-1
            likelihood_yt = np.sum(predicted_prob_st * omega_t[t, :]) # Scalar

            # Store log-likelihood contribution, handling potential log(0)
            if likelihood_yt <= 1e-300: # Use a very small floor
                # warnings.warn(f"Likelihood near zero at t={t + 1}. Check model/data.", RuntimeWarning) # Can be noisy
                LLs[t] = log_lik_floor
                # Reset next state probability to avoid propagation of zeros/NaNs
                pmat[t + 1, :] = 1.0 / k_st
            else:
                LLs[t] = max(np.log(likelihood_yt), log_lik_floor) # Prevent extreme negative LLs

                # Update step: P(S_t | data_{1:t}) = P(S_t | data_{1:t-1}) * p(y_t | S_t) / p(y_t | data_{1:t-1})
                numerator = predicted_prob_st * omega_t[t, :]  # Element-wise product
                pmat[t + 1, :] = numerator / likelihood_yt

                # Normalize updated probabilities to ensure they sum to 1
                pmat[t + 1, :] = np.maximum(pmat[t + 1, :], 0) # Ensure non-negative
                sum_prob = np.sum(pmat[t + 1, :])
                if sum_prob > 1e-10:
                     pmat[t + 1, :] = pmat[t + 1, :] / sum_prob
                else: # If sum is effectively zero (numerical issue), reset to uniform
                     pmat[t + 1, :] = 1.0 / k_st

        # Calculate total negative log-likelihood
        neg_ll = -np.sum(LLs)

        # Check for NaN/Inf in final LL (can happen if LLs contains NaNs/Infs)
        if not np.isfinite(neg_ll):
            warnings.warn(f"Log-likelihood is not finite ({neg_ll}). Optimization might fail.", RuntimeWarning)
            # Return large penalty, add noise to potentially help optimizer escape
            neg_ll = 1e12 + np.random.randn() * 1e6

        # Return results
        if return_details:
            # Ensure pmat is finite before returning
            pmat = np.nan_to_num(pmat, nan=1.0/k_st) # Replace NaN with uniform
            return {
                "pmat": pmat,  # Shape (T+1, k_states)
                "LL": neg_ll,  # Negative Log Likelihood
                "LLs": LLs,   # Vector of log contributions (T,)
                "A": A,       # Transition Matrix
                "g_m": g_m    # State Vol Multipliers
            }
        else:
            # Return only the negative log-likelihood for optimization
            return neg_ll

    def _log_likelihood_objective(self, params, *args):
        """
        Objective function for the optimizer (scipy.optimize.minimize).
        Returns the negative log-likelihood calculated by _calculate_likelihood.
        """
        # Directly call _calculate_likelihood which handles parameter clipping
        return self._calculate_likelihood(params, return_details=False)

    def _smooth_probabilities(self, A, filtered_pmat):
        """
        Calculates smoothed probabilities P(state_t | data_{1:T}) using Kim's smoother.
        Requires the transition matrix (A) and filtered probabilities from Hamilton filter.
        Returns smoothed probabilities matrix (T x k_states).
        """
        T_plus_1, k = filtered_pmat.shape  # T_plus_1 = T_obs + 1
        T_obs = T_plus_1 - 1

        # Check if filtered_pmat contains NaNs - if so, cannot smooth reliably
        if np.any(np.isnan(filtered_pmat)):
             warnings.warn("NaNs detected in filtered probabilities before smoothing. Smoothing may be unreliable. Returning uniform.", RuntimeWarning)
             return np.ones((T_obs, k)) / k

        smoothed_p = np.zeros((T_obs, k))  # Smoothed probs for t=1...T_obs

        # Smoothed prob at T is just the filtered prob at T
        # filtered_pmat[T_obs] contains P(S_T | data_{1:T})
        smoothed_p[T_obs - 1, :] = filtered_pmat[T_obs, :]

        # Pre-calculate predicted probabilities P(S_{t+1} | data_{1:t}) needed for denominator
        predicted_p = np.zeros((T_obs, k))
        for t in range(T_obs):
            # P(S_{t+1} | data_{1:t}) = sum_i P(S_t=i | data_{1:t}) * P(S_{t+1} | S_t=i)
            # filtered_pmat[t+1, :] contains P(S_t | data_{1:t})
            pred_step = filtered_pmat[t + 1, :] @ A # Prediction based on *updated* state t prob
            pred_step = np.maximum(pred_step, 0)
            sum_pred = np.sum(pred_step)
            if sum_pred > 1e-10:
                 predicted_p[t, :] = pred_step / sum_pred # Store P(S_{t+1} | data_{1:t})
            else:
                 predicted_p[t, :] = 1.0 / k # Fallback if prediction is zero

        # Run smoother backward from T-1 down to 0
        # Uses formula: P(S_t|Y_T) = P(S_t|Y_t) .* { A' * [ P(S_{t+1}|Y_T) ./ P(S_{t+1}|Y_t) ] }
        # Where .* is element-wise product, ./ is element-wise division
        for t in range(T_obs - 2, -1, -1):
            # Denominator term: P(S_{t+1} | data_{1:t})
            pred_next = predicted_p[t, :]
            pred_next = np.maximum(pred_next, 1e-100) # Floor denominator

            # Ratio term: P(S_{t+1} | data_{1:T}) / P(S_{t+1} | data_{1:t})
            ratio = smoothed_p[t + 1, :] / pred_next # Element-wise division

            # Update factor: A' * ratio (matrix multiplication)
            update_term = A.T @ ratio # Shape (k,)

            # Smoothed probability: P(S_t | data_{1:T})
            # filtered_pmat[t+1, :] contains P(S_t | data_{1:t})
            smoothed_p[t, :] = filtered_pmat[t + 1, :] * update_term # Element-wise product

            # Normalize smoothed probability
            smoothed_p[t, :] = np.maximum(smoothed_p[t, :], 0)
            sum_smooth = np.sum(smoothed_p[t, :])
            if sum_smooth > 1e-10:
                smoothed_p[t, :] = smoothed_p[t, :] / sum_smooth
            else: # Reset if things go numerically wrong
                smoothed_p[t, :] = 1.0 / k

        # Final check for NaNs/Infs in smoothed probs
        if not np.all(np.isfinite(smoothed_p)):
             warnings.warn("Non-finite values encountered during smoothing. Resetting to uniform.", RuntimeWarning)
             smoothed_p = np.ones((T_obs, k)) / k

        # Return smoothed probabilities for observations t=1...T_obs
        return smoothed_p  # Shape (T_obs, k_states)

    def _predict_volatility(self, P, A, g_m, sigma_unann, h=None):
        """
        Calculates predicted or fitted conditional volatility/variance.
        Uses state probabilities (P), transition matrix (A), multipliers (g_m),
        and base sigma. Handles both in-sample fitting (h=None) and
        h-step ahead forecasting. Returns annualized values.
        """
        # Check for valid inputs
        if P is None or A is None or g_m is None or sigma_unann is None:
            warnings.warn("Missing inputs for volatility prediction.", RuntimeWarning)
            return {"vol": np.nan, "vol_sq": np.nan}

        # Clip sigma
        sigma_unann = np.maximum(sigma_unann, 1e-8)

        # --- Forecasting h steps ahead ---
        if h is not None:
            if not isinstance(h, int) or h < 1:
                raise ValueError("Forecast horizon h must be a positive integer.")
            # P should be the last filtered probability P(S_T | data_{1:T})
            if P.shape[0] != 1:
                 raise ValueError("For forecasting (h>=1), P must be the last probability vector (1 x k_states).")

            # Calculate P(S_{T+h} | data_{1:T}) = P(S_T | data_{1:T}) @ A^h
            try:
                A_h = self._matrix_power(A, h)
            except ValueError as e: # Handle potential errors in matrix power
                 warnings.warn(f"Error calculating matrix power A^{h}: {e}. Returning NaN forecast.", RuntimeWarning)
                 return {"vol": np.array([[np.nan]]), "vol_sq": np.array([[np.nan]])}

            p_hat = P @ A_h  # Shape (1, k_states)
            # Ensure predicted probabilities are valid
            p_hat = np.maximum(p_hat, 0)
            p_hat_sum = np.sum(p_hat)
            if p_hat_sum > 1e-10:
                 p_hat = p_hat / p_hat_sum
            else:
                 p_hat = np.ones_like(p_hat) / p_hat.shape[1] # Uniform if sum is zero


        # --- Calculating fitted values (using smoothed or filtered probs) ---
        else:
            p_hat = P  # Shape (T, k_states)

        # Calculate expected squared multiplier E[g_m^2 | Info] = sum_j P(state=j) * g_m(j)^2
        g_m_squared = np.square(g_m)  # Shape (1, k_states)
        # Matrix multiplication: (T, k) @ (k, 1) -> (T, 1) or (1, k) @ (k, 1) -> (1, 1)
        expected_g_m_sq = p_hat @ g_m_squared.T

        # Calculate unannualized variance: sigma^2 * E[g_m^2]
        vol_sq_unann = (sigma_unann ** 2) * expected_g_m_sq
        vol_sq_unann = np.maximum(vol_sq_unann, 1e-16) # Floor variance to prevent negative sqrt

        # Annualize variance and calculate volatility
        vol_sq_ann = vol_sq_unann * self.n_vol
        vol_ann = np.sqrt(vol_sq_ann)

        # Ensure results are finite
        vol_ann = np.nan_to_num(vol_ann, nan=np.nan, posinf=np.nan, neginf=np.nan)
        vol_sq_ann = np.nan_to_num(vol_sq_ann, nan=np.nan, posinf=np.nan, neginf=np.nan)


        return {
            "vol": vol_ann,     # Annualized Volatility
            "vol_sq": vol_sq_ann # Annualized Variance
        }

    def _calculate_gradient(self, params):
        """
        Calculates the gradient (score) of the log-likelihood function
        with respect to the parameters [m0, b, gammak, sigma_unann]
        using numerical central differences. Required for standard error calculation.
        Returns gradient matrix (T x n_params). Handles kbar=1 case for 'b'.
        """
        n_params = len(params)
        T = self.ret.shape[0]
        grad = np.zeros((T, n_params))

        # Step size for numerical differentiation
        h = 1e-7 # Slightly larger step size can improve stability
        # Calculate absolute step size, respecting bounds (min step h)
        h_vec = np.maximum(np.abs(params) * h, h)

        for i in range(n_params):
            # Skip calculation for 'b' if kbar == 1 (parameter is fixed)
            if self.kbar == 1 and i == 1:
                grad[:, i] = 0.0
                continue

            params_fwd = params.copy()
            params_bwd = params.copy()

            # Apply step
            params_fwd[i] += h_vec[i]
            params_bwd[i] -= h_vec[i]

            # Clip parameters to bounds *after* stepping
            params_fwd = np.clip(params_fwd, self._lb, self._ub)
            params_bwd = np.clip(params_bwd, self._lb, self._ub)

            # Ensure the step actually changed the parameter after clipping
            actual_h = params_fwd[i] - params_bwd[i]
            if actual_h < 1e-12: # If step is negligible (e.g., param at bound)
                 grad[:, i] = 0.0
                 continue # Skip gradient calculation for this param

            # Calculate LLs for forward and backward steps, handle potential errors
            ll_fwd = None
            try:
                details_fwd = self._calculate_likelihood(params_fwd, return_details=True)
                # Check if likelihood calculation failed internally
                if np.isfinite(details_fwd["LL"]):
                     ll_fwd = details_fwd["LLs"]
            except Exception as e:
                # warnings.warn(f"Error calculating forward LL for grad param {i}: {e}. Grad set to 0.", RuntimeWarning) # Can be noisy
                pass # Keep ll_fwd as None

            ll_bwd = None
            try:
                details_bwd = self._calculate_likelihood(params_bwd, return_details=True)
                if np.isfinite(details_bwd["LL"]):
                     ll_bwd = details_bwd["LLs"]
            except Exception as e:
                # warnings.warn(f"Error calculating backward LL for grad param {i}: {e}. Grad set to 0.", RuntimeWarning)
                pass # Keep ll_bwd as None

            # If either calculation failed, set gradient to zero for this param
            if ll_fwd is None or ll_bwd is None:
                grad[:, i] = 0.0
                continue

            # Ensure LLs vectors are finite for subtraction
            ll_fwd = np.nan_to_num(ll_fwd, nan=-700, posinf=0, neginf=-700)
            ll_bwd = np.nan_to_num(ll_bwd, nan=-700, posinf=0, neginf=-700)

            # Calculate central difference derivative for parameter i
            delta_ll = ll_fwd - ll_bwd
            grad[:, i] = delta_ll / actual_h

        # Check for non-finite values in the final gradient matrix
        if not np.all(np.isfinite(grad)):
            warnings.warn("Non-finite values detected in gradient calculation. Replacing with 0.", RuntimeWarning)
            grad = np.nan_to_num(grad, nan=0.0, posinf=0.0, neginf=0.0)

        return grad  # Shape (T, n_params)

    def _calculate_std_errors(self, params):
        """
        Calculates standard errors for model parameters using the Outer Product
        of Gradients (OPG) estimator. SE = sqrt(diag(inv(J))), where J = sum(g_t * g_t').
        Handles potential numerical issues during matrix inversion.
        Returns standard errors for unannualized parameters (n_params x 1).
        """
        n_params = len(params)
        se = np.full(n_params, np.nan) # Initialize with NaNs

        try:
            # Calculate gradient matrix g (T x n_params)
            g = self._calculate_gradient(params)

            # Check if gradient calculation resulted in all zeros (indicates problem)
            if np.all(np.isclose(g, 0)):
                 warnings.warn("Gradient matrix is all zeros. Cannot calculate standard errors.", RuntimeWarning)
                 return se.reshape(-1, 1)

            # Calculate Outer Product of Gradients (OPG) Information Matrix J
            # J = sum_{t=1}^T [ grad_t * grad_t' ] where grad_t is gradient at time t (row vector)
            J = g.T @ g  # Shape (n_params x n_params)

            # Check if J is singular or ill-conditioned before inverting
            # Use condition number or determinant check
            cond_num = np.linalg.cond(J)
            if cond_num > 1e10: # Check if condition number is very large
                 warnings.warn(f"Information matrix J is ill-conditioned (cond={cond_num:.2e}). Regularizing. SEs may be unreliable.", RuntimeWarning)
                 # Add small value to diagonal for regularization
                 cov_matrix = np.linalg.inv(J + np.eye(n_params) * 1e-6)
            else:
                 cov_matrix = np.linalg.inv(J)

            # Extract variances (diagonal elements)
            variances = np.diag(cov_matrix)

            # Check for negative variances which indicate numerical problems
            if np.any(variances < -1e-10): # Allow for tiny negative values due to precision
                 warnings.warn("Negative variances encountered in SE calculation. Taking absolute value.", RuntimeWarning)
                 variances = np.abs(variances)
            # Ensure non-negativity strictly before sqrt
            variances = np.maximum(variances, 0)

            se = np.sqrt(variances)

        except np.linalg.LinAlgError:
            warnings.warn("Could not invert information matrix J (LinAlgError). Standard errors set to NaN.", RuntimeWarning)
            se = np.full(n_params, np.nan)
        except Exception as e:
             warnings.warn(f"An unexpected error occurred during standard error calculation: {e}. Standard errors set to NaN.", RuntimeWarning)
             se = np.full(n_params, np.nan)

        # Ensure SE for 'b' is NaN if kbar == 1
        if self.kbar == 1:
            se[1] = np.nan

        # Return standard errors for the unannualized parameters
        return se.reshape(-1, 1)

    def _calculate_marginal_probabilities(self, p, m0, Mmat):
        """
        Calculates marginal probabilities P(M_k=m0 | data) for each component k.
        Sums the probabilities of states where component k has value m0.

        Parameters:
        -----------
        p : numpy.ndarray (T x k_states) - Smoothed or filtered state probabilities.
        m0 : float - The base multiplier value.
        Mmat : numpy.ndarray (k_states x kbar) - Component value matrix.

        Returns:
        --------
        numpy.ndarray (T x kbar) - Marginal probabilities P(M_k=m0 | data).
        """
        # Input validation
        if p is None or Mmat is None or not np.all(np.isfinite(p)) or not np.all(np.isfinite(Mmat)):
             warnings.warn("Invalid inputs for marginal probability calculation.", RuntimeWarning)
             T_fallback = p.shape[0] if p is not None else 1
             kbar_fallback = Mmat.shape[1] if Mmat is not None else self.kbar
             return np.zeros((T_fallback, kbar_fallback))
        if p.shape[1] != self.k_states:
            raise ValueError(f"Prob matrix columns ({p.shape[1]}) != k_states ({self.k_states}).")
        if Mmat.shape[0] != self.k_states or Mmat.shape[1] != self.kbar:
            raise ValueError(f"Mmat shape ({Mmat.shape}) incorrect. Expected ({self.k_states}, {self.kbar}).")

        # Clip m0 just in case
        m0 = np.clip(m0, self._lb[0], self._ub[0])

        T = p.shape[0]
        m_marginals_m0 = np.zeros((T, self.kbar))

        # Iterate through each component k (from 0 to kbar-1)
        for k in range(self.kbar):
            # Find the indices (rows) of states where the k-th component value is m0
            # Mmat[:, k] gives the value of component k+1 for all states
            states_where_comp_k_is_m0 = np.isclose(Mmat[:, k], m0, atol=1e-5) # Use isclose for float comparison

            # Sum the probabilities (p) of these specific states at each time t
            # p[:, states_where_comp_k_is_m0] selects columns (states) where M_k is m0
            # Summing along axis=1 sums these probabilities for each time t
            m_marginals_m0[:, k] = np.sum(p[:, states_where_comp_k_is_m0], axis=1)

         # Clip results to be valid probabilities [0, 1]
        m_marginals_m0 = np.clip(m_marginals_m0, 0, 1)

        return m_marginals_m0 # P(M_k = m0 | data)


    def _fit(self):
        """
        Fits the MSM model by maximizing the log-likelihood function using
        scipy.optimize.minimize with the L-BFGS-B method (handles bounds).
        Stores estimated parameters, standard errors, likelihood, and other
        results in the `results` dictionary.
        """
        # Define bounds for the optimizer
        bounds = list(zip(self._lb, self._ub))

        # Ensure initial parameters are within bounds before starting optimization
        para0_clipped = np.clip(self._para0, self._lb, self._ub)
        # Ensure b is fixed if kbar=1
        if self.kbar == 1:
            para0_clipped[1] = 1.5

        # Define the objective function (negative log-likelihood)
        objective = self._log_likelihood_objective

        # print("Starting optimization...") # Removed verbose flag dependency
        # Define optimization options
        # ftol/gtol control tolerance for termination based on function value/gradient changes
        opt_options = {'disp': False, 'maxiter': 500, 'ftol': 1e-9, 'gtol': 1e-5}

        opt_result = None # Initialize result
        try:
            opt_result = minimize(
                objective,
                para0_clipped, # Use clipped initial guess (unannualized sigma)
                args=(),        # No extra args needed for objective
                method='L-BFGS-B', # Suitable for bounded problems
                bounds=bounds,
                options=opt_options
            )
            # print(f"Optimization finished: Success={opt_result.success}, Message={opt_result.message}") # Removed verbose

            # Store parameters even if optimization didn't fully converge
            # Clip final parameters just to be safe (should be within bounds anyway)
            self.parameters = np.clip(opt_result.x, self._lb, self._ub)

            if not opt_result.success:
                warnings.warn(f"Optimization did not converge: {opt_result.message}. Parameters may not be optimal.", RuntimeWarning)

        except Exception as e:
             warnings.warn(f"Fatal error during optimization: {e}. Model fitting failed.", RuntimeWarning)
             # Set attributes to indicate failure if optimization crashes
             self.parameters = None # Indicate failure
             self.std_errors = None
             self.log_likelihood = -np.inf
             self.results = {"optim_convergence": False, "optim_message": f"Optimization Error: {e}"}
             return # Exit fit method if optimization crashes

        # --- Post-Optimization Calculations (even if not converged) ---
        # Recalculate likelihood details with the final parameters found
        final_likelihood_details = self._calculate_likelihood(self.parameters, return_details=True)

        # Store positive log-likelihood
        self.log_likelihood = -final_likelihood_details["LL"]

        # --- Calculate Standard Errors ---
        # Calculate SEs based on the parameters found (may be NaN if fit failed badly)
        self.std_errors = self._calculate_std_errors(self.parameters)

        # --- Calculate Smoothed Probabilities ---
        # Need to ensure inputs to smoother are valid
        A_final = final_likelihood_details.get("A")
        pmat_final = final_likelihood_details.get("pmat")
        smoothed_p = None # Initialize
        if A_final is None or pmat_final is None or not np.all(np.isfinite(pmat_final)):
             warnings.warn("Invalid inputs for smoothing after optimization. Smoothed probabilities set to uniform.", RuntimeWarning)
             T_obs = self.ret.shape[0]
             smoothed_p = np.ones((T_obs, self.k_states)) / self.k_states
        else:
             try:
                 smoothed_p = self._smooth_probabilities(A_final, pmat_final) # Should be (T, k)
             except Exception as smooth_e:
                 warnings.warn(f"Error during smoothing: {smooth_e}. Smoothed probabilities set to uniform.", RuntimeWarning)
                 T_obs = self.ret.shape[0]
                 smoothed_p = np.ones((T_obs, self.k_states)) / self.k_states


        # --- Prepare results dictionary ---
        coef = self.parameters.copy() # Unannualized sigma
        se_display = self.std_errors.flatten().copy() if self.std_errors is not None else np.full(len(coef), np.nan) # Flatten SEs

        # Handle kbar=1 case for display
        if self.kbar == 1:
            coef[1] = np.nan
            if len(se_display) > 1: se_display[1] = np.nan

        # Annualize sigma and its standard error for display coefficients
        # SE[f(x)] approx |f'(x)| * SE[x]
        # f(sigma_unann) = sigma_unann * sqrt(n_vol) => f' = sqrt(n_vol)
        scaling_factor = np.sqrt(self.n_vol)
        coef[3] = coef[3] * scaling_factor # Annualize sigma estimate
        # Annualize SE only if it's valid
        if len(se_display) > 3 and np.isfinite(se_display[3]):
             se_display[3] = se_display[3] * scaling_factor
        elif len(se_display) > 3:
             se_display[3] = np.nan # Keep NaN if original SE was NaN

        # Store all results
        self.results = {
            "LL": self.log_likelihood,
            "LLs": final_likelihood_details.get("LLs"), # Shape (T,)
            # Filtered probabilities P(S_t | data_{1:t}) for t=1...T
            "filtered_probabilities": final_likelihood_details.get("pmat")[1:, :] if final_likelihood_details.get("pmat") is not None else None, # Shape (T, k)
            "smoothed_probabilities": smoothed_p,  # Shape (T, k_states)
            "transition_matrix": final_likelihood_details.get("A"),
            "state_vol_multipliers": final_likelihood_details.get("g_m"),
            "component_matrix": self._calculate_component_matrix(self.parameters[0]), # Use m0 from estimated params
            "optim_message": opt_result.message if opt_result else "Optimization failed before result",
            "optim_convergence": opt_result.success if opt_result else False,
            "optim_iter": opt_result.nit if opt_result and hasattr(opt_result, 'nit') else -1, # L-BFGS-B uses 'nit'
            "parameters": self.parameters,  # [m0, b, gammak, sigma_unann]
            "std_errors": self.std_errors,  # SEs for unannualized sigma (n_params x 1)
            "coefficients": coef,           # Coefs for display (annualized sigma) (n_params,)
            "se_display": se_display        # SEs for display (annualized sigma) (n_params,)
        }
        self.coef_names = ["m0", "b", "gammak", "sigma_annualized"]


    def summary(self):
        """
        Prints a summary table of the estimated model parameters, standard errors,
        t-values, and p-values. Uses annualized sigma for display.
        """
        # Check if model has been fitted successfully
        if self.parameters is None or not self.results or not self.results.get('optim_convergence', False):
            # Added check for convergence status
            print("Model has not been fitted successfully or did not converge.")
            return None

        # Retrieve coefficients and SEs prepared for display
        coef = self.results.get("coefficients", np.full(4, np.nan))
        se = self.results.get("se_display", np.full(4, np.nan)) # Should be flat
        log_likelihood = self.results.get("LL", np.nan)
        convergence = self.results.get("optim_convergence", False)
        iterations = self.results.get("optim_iter", "N/A")
        message = self.results.get("optim_message", "N/A")

        # Calculate t-values and p-values
        tval = np.full_like(coef, np.nan)
        pval = np.full_like(coef, np.nan)

        # Calculate only where SE is valid (not NaN and not near zero)
        valid_se_mask = ~np.isnan(se) & (np.abs(se) > 1e-12)
        tval[valid_se_mask] = coef[valid_se_mask] / se[valid_se_mask]

        # Degrees of freedom: T - number of estimated parameters
        # Count non-NaN coefficients as estimated parameters
        n_estimated_params = np.sum(~np.isnan(coef))
        T = len(self.ret)
        df = T - n_estimated_params

        # Calculate p-values using t-distribution if df > 0
        if df > 0:
            # Calculate only where t-value is valid
            valid_tval_mask = ~np.isnan(tval)
            pval[valid_tval_mask] = 2 * (1 - stats.t.cdf(np.abs(tval[valid_tval_mask]), df=df))
        else:
            warnings.warn("Degrees of freedom <= 0. Cannot calculate p-values.", RuntimeWarning)

        # --- Print Summary ---
        print("*" * 76)
        print(f"  Markov Switching Multifractal Model (MSM) - kbar={self.kbar}")
        print("*" * 76)
        print(f"  Log-Likelihood: {log_likelihood:.4f}")
        print(f"  Observations:   {T}")
        print(f"  Optimization:   Converged={convergence}, Iterations={iterations}")
        print(f"  Message:        {message}")
        print("-" * 76)

        # Create DataFrame for summary table
        summary_data = {
            "Estimate": coef,
            "Std Error": se,
            "t-value": tval,
            "p-value": pval
        }
        # Use coef_names stored in self, handle case where it might not exist
        coef_names = getattr(self, 'coef_names', ["m0", "b", "gammak", "sigma_annualized"])
        summary_df = pd.DataFrame(summary_data, index=coef_names)

        # Format output nicely
        with pd.option_context('display.float_format', '{:,.4f}'.format):
            print(summary_df.to_string(na_rep='NaN')) # Use to_string to better handle NaNs
        print("-" * 76)

        return summary_df

    def predict(self, h=None):
        """
        Generates fitted values (h=None) or h-step ahead forecasts.

        Parameters:
        -----------
        h : int, optional
            Forecast horizon in periods (matching data frequency).
            - If None (default): Returns fitted conditional volatility based on
              smoothed probabilities (most likely).
            - If h >= 1: Returns h-step ahead volatility forecast based on the
              last available filtered probability.

        Returns:
        --------
        dict {'vol': ndarray, 'vol_sq': ndarray}
            Annualized conditional volatility and variance (fitted or forecast).
            Returns NaNs if model fitting failed or inputs are invalid.
        """
        # Check if model fitting was successful
        if self.parameters is None or not self.results or not self.results.get('optim_convergence', False):
             # print("Model has not been fitted successfully. Cannot predict.") # Can be noisy
             # Return dict with NaNs of expected shape
             T = len(self.ret) if hasattr(self, 'ret') and self.ret is not None else 1
             shape_out = (1, 1) if h is not None else (T, 1)
             return {
                 "vol": np.full(shape_out, np.nan),
                 "vol_sq": np.full(shape_out, np.nan)
             }

        # Retrieve necessary components from results
        sigma_unann = self.parameters[3]
        A = self.results.get("transition_matrix")
        g_m = self.results.get("state_vol_multipliers")
        filtered_p = self.results.get("filtered_probabilities") # Shape (T, k)
        smoothed_p = self.results.get("smoothed_probabilities") # Shape (T, k)

        # --- Input Validation for Prediction ---
        if A is None or g_m is None:
             warnings.warn("Transition matrix or multipliers not available. Cannot predict.", RuntimeWarning)
             T = len(self.ret); shape_out = (1, 1) if h is not None else (T, 1)
             return {"vol": np.full(shape_out, np.nan), "vol_sq": np.full(shape_out, np.nan)}


        # Determine which probability matrix to use based on h
        if h is None: # Fitted values -> Use smoothed probabilities if available
            P_in = smoothed_p
            if P_in is None or not np.all(np.isfinite(P_in)):
                 warnings.warn("Smoothed probabilities invalid/unavailable for fitting. Returning NaNs.", RuntimeWarning)
                 T = len(self.ret)
                 return {"vol": np.full((T,1), np.nan), "vol_sq": np.full((T,1), np.nan)}
        else: # Forecasting -> Use last row of filtered probabilities
            if not isinstance(h, int) or h < 1:
                 raise ValueError("Forecast horizon h must be a positive integer.")
            # Check if filtered probabilities are valid
            if filtered_p is None or filtered_p.shape[0] == 0 or not np.all(np.isfinite(filtered_p[-1,:])):
                 warnings.warn("Filtered probabilities invalid/unavailable for forecasting. Returning NaNs.", RuntimeWarning)
                 return {"vol": np.array([[np.nan]]), "vol_sq": np.array([[np.nan]])}
            # Use the last row P(S_T | data_{1:T})
            P_in = filtered_p[-1:, :] # Shape (1, k)

        # Call the internal prediction function
        return self._predict_volatility(P_in, A, g_m, sigma_unann, h=h)


    def plot(self, plot_type="vol", use_smoothed=True):
        """
        Plots fitted conditional volatility or variance against a proxy
        (absolute or squared returns).

        Parameters:
        -----------
        plot_type : str, 'vol' or 'volsq'
            Whether to plot volatility or variance.
        use_smoothed : bool
            If True, uses smoothed probabilities for fitted values.
            If False, uses filtered probabilities.
        """
        # Check model state
        if self.parameters is None or not self.results:
            print("Model has not been fitted or fitting failed. Cannot plot.")
            return
        if not self.results.get('optim_convergence', False):
             print("Model did not converge. Plotting may be based on non-optimal parameters.")


        if plot_type not in ["vol", "volsq"]:
            raise ValueError("plot_type must be either 'vol' or 'volsq'")

        # --- Get Fitted Values ---
        # Retrieve necessary components
        sigma_unann = self.parameters[3]
        A = self.results.get("transition_matrix")
        g_m = self.results.get("state_vol_multipliers")

        # Select probabilities based on use_smoothed flag
        if use_smoothed:
            P_fit = self.results.get("smoothed_probabilities")
            title_suffix = "(Smoothed)"
            if P_fit is None or not np.all(np.isfinite(P_fit)):
                 print("Smoothed probabilities not available or invalid. Cannot plot.")
                 return
        else:
            P_fit = self.results.get("filtered_probabilities")
            title_suffix = "(Filtered)"
            if P_fit is None or not np.all(np.isfinite(P_fit)):
                 print("Filtered probabilities not available or invalid. Cannot plot.")
                 return

        # Calculate fitted values using the internal method (h=None)
        pred = self._predict_volatility(P_fit, A, g_m, sigma_unann, h=None)

        # Check if prediction was successful
        if pred is None or 'vol' not in pred or np.all(np.isnan(pred['vol'])):
             print("Fitted volatility could not be calculated. Cannot plot.")
             return

        # Ensure returns have the same length as fitted vol
        T_fit = pred['vol'].shape[0]
        # Use self.ret which should be (T, 1)
        returns_plot = self.ret[:T_fit, 0] # Match length and get 1D array

        # --- Create Plot ---
        plt.figure(figsize=(12, 6))

        if plot_type == "vol":
            fitted_values = pred["vol"][:, 0] # Get 1D array
            # Calculate annualized absolute returns as proxy
            data_proxy = np.abs(returns_plot * np.sqrt(self.n_vol))
            proxy_label = "Annualized Abs Returns"
            plot_title = f"Fitted Conditional Volatility vs Absolute Returns {title_suffix}"
            ylabel = "Annualized Volatility"
        else: # volsq
            fitted_values = pred["vol_sq"][:, 0] # Get 1D array
            # Calculate annualized squared returns as proxy
            data_proxy = np.square(returns_plot * np.sqrt(self.n_vol))
            proxy_label = "Annualized Squared Returns"
            plot_title = f"Fitted Conditional Variance vs Squared Returns {title_suffix}"
            ylabel = "Annualized Variance"

        # Plot fitted values and proxy
        plt.plot(fitted_values, label=f"Fitted Annualized {plot_type.capitalize()} {title_suffix}", color='blue', linewidth=1.5)
        plt.plot(data_proxy, label=proxy_label, alpha=0.5, color='orange', linestyle='-', linewidth=0.8)
        plt.title(plot_title)
        plt.ylabel(ylabel)
        plt.xlabel("Time Period")
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        plt.show()


    def decompose(self, use_smoothed=True):
        """
        Decomposes volatility into contributions from each frequency component M_k.
        Calculates E[M_k | data] = m0*P(M_k=m0|data) + (2-m0)*P(M_k=2-m0|data).

        Parameters:
        -----------
        use_smoothed : bool, optional (default True)
            Use smoothed or filtered probabilities.

        Returns:
        --------
        numpy.ndarray (T x kbar) or None if failed.
        """
        # Check model state
        if self.parameters is None or not self.results:
            print("Model has not been fitted or fitting failed.")
            return None
        # Allow decomposition even if not converged, but warn
        if not self.results.get('optim_convergence', False):
             print("Warning: Model did not converge. Decomposition based on non-optimal parameters.")


        m0 = self.parameters[0]
        Mmat = self.results.get("component_matrix") # k_states x kbar
        if Mmat is None:
             print("Component matrix not available. Cannot decompose.")
             return None

        # Select probabilities
        if use_smoothed:
            P = self.results.get("smoothed_probabilities") # T x k_states
            prob_type = "Smoothed"
            if P is None or not np.all(np.isfinite(P)):
                 print("Smoothed probabilities invalid/unavailable. Cannot decompose.")
                 return None
        else:
            P = self.results.get("filtered_probabilities") # T x k_states
            prob_type = "Filtered"
            if P is None or not np.all(np.isfinite(P)):
                 print("Filtered probabilities invalid/unavailable. Cannot decompose.")
                 return None

        # Calculate marginal probabilities P(M_k = m0 | data) -> shape (T, kbar)
        try:
             p_m0_marginals = self._calculate_marginal_probabilities(P, m0, Mmat)
        except Exception as e:
             print(f"Error calculating marginal probabilities: {e}")
             return None

        # Calculate E[M_k | data] = m0 * P(M_k=m0) + m1 * (1 - P(M_k=m0))
        m1 = 2.0 - m0
        expected_M_k = m0 * p_m0_marginals + m1 * (1.0 - p_m0_marginals)

        # Ensure result is finite
        if not np.all(np.isfinite(expected_M_k)):
             warnings.warn("Non-finite values in expected components. Replacing with average.", RuntimeWarning)
             avg_val = m0 * 0.5 + m1 * 0.5 # Default expectation if probs are 0.5
             expected_M_k = np.nan_to_num(expected_M_k, nan=avg_val, posinf=m0, neginf=m1)

        return expected_M_k  # Shape (T, kbar)

    def plot_components(self, use_smoothed=True):
        """
        Plots the decomposed volatility components E[M_k | data] over time.

        Parameters:
        -----------
        use_smoothed : bool, optional (default True)
            Passed to the decompose method.
        """
        if self.parameters is None or not self.results:
            print("Model has not been fitted or fitting failed.")
            return

        # Get decomposed components
        expected_M_k = self.decompose(use_smoothed=use_smoothed)
        if expected_M_k is None:
            print("Decomposition failed. Cannot plot components.")
            return

        prob_type = "Smoothed" if use_smoothed else "Filtered"
        T, num_components = expected_M_k.shape

        # Adjust plotting layout based on number of components
        if num_components == 0:
             print("No components to plot (kbar=0?).")
             return
        elif num_components == 1:
             fig, ax = plt.subplots(1, 1, figsize=(12, 4))
             axes = [ax] # Make it iterable
        else:
             # Create subplots, sharing the x-axis
             fig, axes = plt.subplots(num_components, 1, figsize=(12, 2.5 * num_components), sharex=True)

        fig.suptitle(f"Expected Volatility Components E[M_k | Data] ({prob_type})", fontsize=14)

        # Get m0 and m1 for plotting horizontal lines
        m0 = self.parameters[0]
        m1 = 2.0 - m0

        # Plot each component's expectation
        for k in range(num_components):
            ax = axes[k]
            # Plot E[M_{k+1} | Data]
            ax.plot(expected_M_k[:, k], label=f"E[M$_{k + 1}$ | Data]", linewidth=1.5)
            # Add horizontal lines for m0 and m1
            ax.axhline(m0, color='r', linestyle='--', alpha=0.7, label=f'$m_0 \\approx {m0:.3f}$')
            ax.axhline(m1, color='g', linestyle='--', alpha=0.7, label=f'$m_1 \\approx {m1:.3f}$')

            # Set titles and labels
            ax.set_title(f"Component M$_{k + 1}$")
            ax.set_ylabel("$E[M_k]$")
            ax.grid(True, linestyle='--', alpha=0.6)
            ax.legend(loc='best')
            # Set y-limits for better visualization
            ax.set_ylim(min(m0, m1) - 0.1, max(m0, m1) + 0.1)

        # Set x-label only on the bottom-most plot
        axes[-1].set_xlabel("Time Period")

        # Adjust layout to prevent title overlap
        plt.tight_layout(rect=[0, 0.03, 1, 0.96]) # [left, bottom, right, top]
        plt.show()


# =============================================================================
# Evaluation Pipeline Script
# =============================================================================

# --- Configuration ---
# Path to your 5-minute data CSV
# >>>>>>>> IMPORTANT: Update this path <<<<<<<<<<<
root = Path().resolve().parent # Assuming script is one level down from project root
data_path = root / 'data' / 'csi2007.csv' # Path from user's code
data_file = data_path

# MSM Model Parameters
# --- FIX 1: Reduce KBAR significantly ---
KBAR = 3 # Start with 3 components (8 states), try 2 if this still fails
# --- FIX 2: Use standard annualization factor ---
ANNUALIZATION_FACTOR = 252 # Trading days in a year

# Rolling Window Parameters
INITIAL_ESTIMATION_YEARS = 3 # Years of data for the first fit
EVALUATION_START_YEAR = 2008 # Start forecasting from this year
MIN_ROLLING_WINDOW_DAYS = 200 # Minimum number of days required in the rolling window

# --- Helper Functions ---

def calculate_realized_volatility(df_5min_returns, annualization_factor=252):
    """Calculates daily realized variance and volatility from 5-min returns."""
    if not isinstance(df_5min_returns.index, pd.DatetimeIndex):
        raise ValueError("Input DataFrame must have a DatetimeIndex.")
    df_copy = df_5min_returns.copy()
    # Ensure log returns are numeric and handle potential infinities
    df_copy['log_ret_5min'] = pd.to_numeric(df_copy['log_ret_5min'], errors='coerce')
    df_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_copy.dropna(subset=['log_ret_5min'], inplace=True) # Drop NaNs again after coercion/inf handling

    df_copy['log_ret_5min_sq'] = df_copy['log_ret_5min']**2
    # Ensure sum only happens if there are valid squared returns for the day
    realized_variance_daily = df_copy['log_ret_5min_sq'].resample('D').sum(min_count=1) # Require at least 1 valid sq return
    realized_volatility_daily = np.sqrt(realized_variance_daily * annualization_factor)
    realized_volatility_daily.name = 'realized_volatility'
    return realized_volatility_daily

def calculate_daily_returns(df_5min, price_col='close'):
    """Calculates daily log returns from 5-min close prices."""
    if not isinstance(df_5min.index, pd.DatetimeIndex):
        raise ValueError("Input DataFrame must have a DatetimeIndex.")
    # Ensure price column is numeric
    df_5min[price_col] = pd.to_numeric(df_5min[price_col], errors='coerce')
    # Use 'B' frequency to only get business day closes
    daily_close = df_5min[price_col].resample('B').last()
    # Forward fill missing daily closes (e.g., holidays) before calculating returns
    daily_close = daily_close.ffill()
    daily_log_returns = np.log(daily_close).diff()
    daily_log_returns.name = 'log_ret_daily'
    return daily_log_returns

def evaluate_forecasts(comparison_df):
    """Calculates and prints evaluation metrics."""
    required_cols = ['realized_volatility', 'msm_forecast']
    if not all(col in comparison_df.columns for col in required_cols):
        print("Error: Missing required columns for evaluation.")
        return None
    eval_df = comparison_df[required_cols].dropna()
    if eval_df.empty or len(eval_df) < 2: # Need at least 2 points for regression
        print("No overlapping data or insufficient data available for evaluation after dropping NaNs.")
        return None

    realized_vol = eval_df['realized_volatility']
    forecast_vol = eval_df['msm_forecast']

    # Add check for non-finite values before metrics
    finite_mask = np.isfinite(realized_vol) & np.isfinite(forecast_vol)
    if not finite_mask.all():
        print("Warning: Non-finite values found in realized or forecast volatility. Filtering before evaluation.")
        realized_vol = realized_vol[finite_mask]
        forecast_vol = forecast_vol[finite_mask]
        if len(realized_vol) < 2:
             print("Insufficient finite data points remaining for evaluation.")
             return None


    # --- Metrics ---
    if len(realized_vol) == 0 or len(forecast_vol) == 0:
        print("Cannot calculate metrics: No valid data points.")
        return eval_df

    try:
        mse = mean_squared_error(realized_vol, forecast_vol)
        mae = np.mean(np.abs(realized_vol - forecast_vol))
        rmse = np.sqrt(mse)
        epsilon = 1e-12
        realized_var = np.maximum(realized_vol, 0)**2 + epsilon
        forecast_var = np.maximum(forecast_vol, 0)**2 + epsilon
        forecast_var = np.maximum(forecast_var, epsilon)
        valid_qlike_mask = (forecast_var > 0) & (realized_var >= 0)
        if np.sum(valid_qlike_mask) > 0:
            qlike = np.mean(np.log(forecast_var[valid_qlike_mask]) + realized_var[valid_qlike_mask] / forecast_var[valid_qlike_mask])
        else:
            qlike = np.nan

        print("\n--- Forecast Evaluation Metrics ---")
        print(f"  RMSE: {rmse:.6f}")
        print(f"  MAE:  {mae:.6f}")
        print(f"  MSE:  {mse:.6f}")
        print(f"  QLIKE:{qlike:.4f} (Lower is better)")

    except Exception as metric_e:
        print(f"Error calculating basic metrics: {metric_e}")
        return eval_df


    # --- Mincer-Zarnowitz Regression ---
    if np.std(forecast_vol) > 1e-9 and len(forecast_vol) > 1:
        X = sm.add_constant(forecast_vol)
        y = realized_vol
        try:
            mz_model = sm.OLS(y, X).fit()
            print("\n--- Mincer-Zarnowitz Regression (Realized ~ const + Forecast) ---")
            print(mz_model.summary())
        except Exception as e:
            print(f"\nCould not perform Mincer-Zarnowitz regression: {e}")
    else:
        print("\nSkipping Mincer-Zarnowitz regression: Insufficient variation in forecasts or too few data points.")


    # --- Plotting ---
    try:
        plt.figure(figsize=(14, 7))
        # Use original eval_df index but filtered data for plotting
        plot_index = eval_df.index[finite_mask] if 'finite_mask' in locals() and finite_mask.any() else eval_df.index
        plt.plot(plot_index, realized_vol, label='Realized Volatility (Ground Truth)', alpha=0.7, linewidth=1.5, color='black')
        plt.plot(plot_index, forecast_vol, label=f'MSM Forecast (k={KBAR})', alpha=0.8, linewidth=1.2, color='red')
        plt.title('MSM Daily Volatility Forecast vs. Realized Volatility')
        plt.xlabel('Date')
        plt.ylabel('Annualized Volatility')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()
    except Exception as plot_e:
        print(f"Error during plotting: {plot_e}")


    return eval_df

# --- Main Pipeline ---

# 1. Load and Prepare 5-minute Data
print(f"Loading 5-minute data from: {data_file}...")
try:
    df_5min = pd.read_csv(
        data_file,
        index_col='datetime',
        parse_dates=True,
    )
    if df_5min.empty: print(f"Error: DataFrame is empty after loading data from {data_file}."); sys.exit(1)
    df_5min.sort_index(inplace=True)
    print(f"Loaded {len(df_5min)} records.")
    print(f"Data range: {df_5min.index.min()} to {df_5min.index.max()}")
    required_cols_5min = ['close']
    if not all(col in df_5min.columns for col in required_cols_5min):
         missing_cols = [col for col in required_cols_5min if col not in df_5min.columns]
         print(f"Error: Missing required columns in loaded 5-min data: {missing_cols}"); sys.exit(1)
except FileNotFoundError: print(f"Error: Data file not found at {data_file}"); raise
except Exception as e: print(f"Error loading or performing initial checks on data: {e}"); raise

# 2. Calculate 5-min Returns
print("Calculating 5-minute log returns...")
if 'close' not in df_5min.columns: print("Error: 'close' column not found."); sys.exit(1)
df_5min['close'] = pd.to_numeric(df_5min['close'], errors='coerce')
df_5min.dropna(subset=['close'], inplace=True)
df_5min['log_ret_5min'] = np.log(df_5min['close']).diff()
df_5min = df_5min.dropna(subset=['log_ret_5min'])

# 3. Calculate Daily Realized Volatility (Ground Truth)
print("Calculating daily realized volatility...")
daily_rv = calculate_realized_volatility(df_5min, ANNUALIZATION_FACTOR)

# 4. Calculate Daily Returns (for MSM fitting)
print("Calculating daily log returns...")
daily_returns = calculate_daily_returns(df_5min)

# 5. Align Data into a Daily DataFrame
print("Aligning daily data...")
df_daily = pd.DataFrame(index=daily_returns.index)
df_daily = df_daily.join(daily_returns).join(daily_rv)
df_daily = df_daily.dropna(subset=['log_ret_daily', 'realized_volatility'])
df_daily = df_daily[np.isfinite(df_daily['log_ret_daily'])]
print(f"Created daily DataFrame with {len(df_daily)} trading days.")
print(f"Daily data range: {df_daily.index.min().date()} to {df_daily.index.max().date()}")

# 6. Rolling Window Forecasting Setup
print("\nSetting up rolling window forecasting...")
try:
    initial_estimation_start_date = df_daily.index.min()
    initial_estimation_end_date_target = initial_estimation_start_date + pd.DateOffset(years=INITIAL_ESTIMATION_YEARS) - pd.Timedelta(days=1)
    valid_end_dates = df_daily.index[df_daily.index <= initial_estimation_end_date_target]
    if valid_end_dates.empty: raise ValueError("Cannot determine initial estimation end date.")
    initial_estimation_end_date = valid_end_dates[-1]

    initial_window_data = df_daily.loc[initial_estimation_start_date:initial_estimation_end_date]
    initial_window_returns = initial_window_data['log_ret_daily'].dropna().values
    initial_window_size_days = len(initial_window_returns) # Use actual returns count
    print(f"Initial estimation window size: {initial_window_size_days} days")

    eval_start_lookup = df_daily.index[df_daily.index >= f"{EVALUATION_START_YEAR}-01-01"]
    if eval_start_lookup.empty: raise ValueError(f"No data found starting from {EVALUATION_START_YEAR}")
    evaluation_start_date = eval_start_lookup[0]
    evaluation_dates = df_daily.loc[evaluation_start_date:].index
    print(f"Initial estimation period: {initial_estimation_start_date.date()} to {initial_estimation_end_date.date()}")
    print(f"Evaluation period: {evaluation_start_date.date()} to {df_daily.index.max().date()} ({len(evaluation_dates)} days)")

    if initial_window_size_days < MIN_ROLLING_WINDOW_DAYS:
        print(f"Warning: Initial estimation window ({initial_window_size_days} days) is smaller than MIN_ROLLING_WINDOW_DAYS ({MIN_ROLLING_WINDOW_DAYS}).")

except (IndexError, ValueError) as e: print(f"Error setting up estimation/evaluation periods: {e}. Check data range and EVALUATION_START_YEAR."); raise
except Exception as e: print(f"Unexpected error setting up estimation/evaluation periods: {e}"); raise

# --- DEBUG: Test Fit on Initial Window ---
print("\n--- Attempting fit on INITIAL estimation window ---")
if len(initial_window_returns) >= MIN_ROLLING_WINDOW_DAYS and np.all(np.isfinite(initial_window_returns)):
    try:
        print(f"Fitting MSM(k={KBAR}) on initial window ({len(initial_window_returns)} days)...")
        # Pass necessary args, removed verbose
        initial_model = MSM(ret=initial_window_returns, kbar=KBAR, n_vol=ANNUALIZATION_FACTOR)
        if initial_model.parameters is None or not initial_model.results.get('optim_convergence', False):
            print("--- Initial Fit FAILED or did not converge ---")
            if hasattr(initial_model, 'results') and initial_model.results:
                 print(f"   Convergence: {initial_model.results.get('optim_convergence', 'N/A')}")
                 print(f"   Message: {initial_model.results.get('optim_message', 'N/A')}")
            else:
                 print("   MSM results object not available.")
        else:
            print("--- Initial Fit SUCCESSFUL ---")
            print(f"   LogLik: {initial_model.log_likelihood:.4f}")
            print(f"   Params (unannualized sigma): {initial_model.parameters}")
    except Exception as initial_e:
        print(f"--- ERROR during initial fit attempt: {type(initial_e).__name__} - {initial_e} ---")
        print(traceback.format_exc())
else:
    print(f"--- Skipping initial fit test: Window size {len(initial_window_returns)} < {MIN_ROLLING_WINDOW_DAYS} or non-finite data ---")
# --- End DEBUG Block ---


print("\nStarting rolling window forecasting loop...")
msm_forecasts = []
forecast_dates = []
failed_fits = 0 # Counter for failed MSM fits
start_time = time.time()

# Use tqdm for progress bar
for i in tqdm(range(len(evaluation_dates)), desc="Rolling Forecast"):
    forecast_for_date = evaluation_dates[i]
    forecast_value = np.nan # Default forecast value if anything fails

    try:
        # --- Window Selection ---
        window_end_idx_loc = df_daily.index.get_loc(forecast_for_date) - 1
        if window_end_idx_loc < 0: continue

        window_start_idx_loc = max(0, window_end_idx_loc - initial_window_size_days + 1)
        current_window_data = df_daily.iloc[window_start_idx_loc : window_end_idx_loc + 1]
        current_returns = current_window_data['log_ret_daily'].dropna().values

        # --- Check Window Size ---
        if len(current_returns) < MIN_ROLLING_WINDOW_DAYS:
            msm_forecasts.append(np.nan)
            forecast_dates.append(forecast_for_date)
            continue

        # --- Attempt MSM Fit ---
        rolling_model = None
        if not np.all(np.isfinite(current_returns)):
             failed_fits += 1
             forecast_value = np.nan
        else:
            # --- Fit Model (removed verbose) ---
            rolling_model = MSM(ret=current_returns, kbar=KBAR, n_vol=ANNUALIZATION_FACTOR)

            # --- Check Fit Success ---
            if rolling_model.parameters is None or not rolling_model.results.get('optim_convergence', False):
                failed_fits += 1
                forecast_value = np.nan
            else:
                # --- Attempt Prediction ---
                prediction = rolling_model.predict(h=1)
                if prediction and 'vol' in prediction and not np.isnan(prediction['vol'][0, 0]):
                    forecast_value = prediction['vol'][0, 0]
                else:
                    forecast_value = np.nan # Ensure NaN if predict fails

    except Exception as e:
        # --- Log the specific error for this iteration ---
        if failed_fits < 10: # Only print first few errors
             print(f"\nERROR during loop for {forecast_for_date.date()}: {type(e).__name__} - {e}")
        elif failed_fits == 10:
             print("\n...(suppressing further error messages from loop)...")
        failed_fits += 1
        forecast_value = np.nan

    # --- Store Results for this date ---
    msm_forecasts.append(forecast_value)
    forecast_dates.append(forecast_for_date)


end_time = time.time()
print(f"\nRolling forecast loop finished in {end_time - start_time:.2f} seconds.")
if failed_fits > 0:
    print(f"Warning: The MSM model fit failed, did not converge, or encountered an error for {failed_fits} out of {len(evaluation_dates)} windows.")

# 7. Combine Forecasts and Evaluate
print("Combining forecasts with daily data...")
if not forecast_dates: print("Error: No forecast dates were generated."); sys.exit(1)
forecast_series = pd.Series(msm_forecasts, index=pd.Index(forecast_dates, name='datetime'), name='msm_forecast')
if forecast_series.index.has_duplicates:
    print("Warning: Duplicate forecast dates found. Keeping last."); forecast_series = forecast_series[~forecast_series.index.duplicated(keep='last')]
df_daily_eval = pd.merge(df_daily, forecast_series, left_index=True, right_index=True, how='left')
df_eval_period = df_daily_eval.loc[evaluation_start_date:].copy()
valid_forecast_count = df_eval_period['msm_forecast'].count()
print(f"Generated {valid_forecast_count} valid forecasts out of {len(df_eval_period)} evaluation days.")

if valid_forecast_count > 0:
    evaluation_results_df = evaluate_forecasts(df_eval_period)
    if evaluation_results_df is not None: print("\n--- Sample of Forecasts vs. Realized Volatility ---\n", evaluation_results_df.head())
    else: print("\nEvaluation could not be completed.")
else: print("\nNo valid forecasts were generated, skipping evaluation.")

print("\nPipeline finished.")